# 사전 준비

In [ ]:
!pip install transformers

# 데이터 불러오기

In [80]:
import pandas as pd

interview_data = pd.read_csv("/content/drive/Othercomputers/내 컴퓨터/Interview_Writer/Interview_Data.csv")

In [ ]:
interview_data.head(3)

,Question,Answer
0,특별히 회사를 선택한 이유와 입사 루 포부에 대해 기술해주십시오.,"대학교 1학년 2학기, 컴퓨터 프로그래밍 및 실습수업을 통해 프로그래밍에 관심을 가..."
1,지원직무를 선택한 이유와 해당직무에 본인이 적임자라고 생각하는 이유를 기술해주십시오.,"대학교 2학년 1학기, 프로그래밍언어 수업을 통해 영상처리를 처음 접하게 되었습니다..."
2,본인이 다니고 싶은 회사와 동료들은 어떤 모습인지 기술해주십시오.,제가 다니고 싶은 회사는 개인 역량을 강화하는데 도움을 주는 회사입니다. 회사에 취...


In [ ]:
interview_data.tail(3)

,Question,Answer
101,본인 성격의 장점과 단점을 기재하여 주시기 바랍니다.,"차분하고 꼼꼼한 성격을 가진 저는 항상 주어진 일에 최선을 다하며, 주위 사람들로부..."
102,본인의 가장 기뻤던 성공경험 또는 가장 쓰라렸던 실패경험을 기재해주시기 바랍니다.,"저는 운동을 좋아합니다. 친구의 추천으로 인해서 자전거를 타기 시작했고, 그 결과 ..."
103,당사에 지원한 동기와 입사 후의 경력개발 계획을 기재해주시기 바랍니다.,저의 Vision은 Programmer입니다. Vision을 이루기 위해 대학 진학...


# 텍스트 전처리

In [ ]:
!pip install soynlp

In [4]:
# 회사이름 리스트를 정규표현식으로 변경
file_path = '/content/drive/Othercomputers/내 컴퓨터/Interview_Writer/companies_name.txt'

with open(file_path) as f:
    names_list = f.read().splitlines()

for i in range(0, len(names_list)):
    if i == 0:
        names_list[i] = str('(' + names_list[i])
    elif i == (len(names_list)-1):
        names_list[i] = str('|' + names_list[i] + ')')
    else:
        names_list[i] = str('|' + names_list[i])

names_str = ''.join(names_list)

print(names_str)

(\x28주\x29동부|AKIS|CJ올리브네트웍스|DB아이앤씨|GS ITM|IBK시스템|KB데이터시스템|KT|KT DS|LG CNS|LG 유플러스|LS아이티씨|NDS|NH농협은행|SK|SK C&C|교보정보통신|농협정보시스템|동국시스템즈|동양네트웍스|동양시스템즈|두산디지털이노베이션|롯데백화점|롯데정보통신|롯데정보통신|메가존클라우드|메타넷대우정보|미라콤아이앤씨|베스핀글로벌|뷰웍스|비즈테크파트너스|삼성SDS|삼성전자|섹타나인|신세계아이앤씨|신한DS|신한금융그룹|신한카드|쌍용정보통신|쓱|아시아나IDT|우리에프아이에스|우리은행|유호스트|줌인터넷|카카오엔터프라이즈|케이뱅크|코오롱베니트|태광|티맥스소프트|티시스|포스코ICT|하나금융티아이|한국데이터베이스진흥원|한진정보통신|한화시스템/ICT|현대HDS|현대IT&E|현대모비스|현대아이파크몰|현대오토에버|현대오토에버|휴머스온)


In [5]:
import re
from soynlp.normalizer import repeat_normalize

pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
whitespace_symbol = re.compile(r'[\t\r\n\f\v]+')

In [6]:
def text_preprocess(x):
    x = pattern.sub(' ', x)                     # 일반적으로 사용하는 특수문자, 영어, 한글제외 공백으로 치환
    x = whitespace_symbol.sub(' ', x)           # space(공백문자)를 제외한 whitespace 문자를 공백으로 치환
    x = re.sub(names_str, 'XX', x)
    x = x.strip()                               # 문자의 시작과 끝에서 공백제거
    x = repeat_normalize(x, num_repeats=2)      # 반목되는 문자의 축약 횟수 2개로 줄임
    return x

In [ ]:
interview_data['Answer'][16]

"OOOOOO에서 주관하는 '제O기 사회리더 대학생 멘토링' 활동 때 팀워크를 발휘하여 공동의 목표 달성에 이바지한 경험이 있습니다. 저희는 고객과 헤어샵 사이 중개 플랫폼을 만들어보자는 목표를 설정하였습니다. 프로젝트에 고객수요조사, 플랫폼 설계 등 혼자 하기에는 무리가 있어서 팀을 구성하였고 저를 포함 5명이 팀을 이루게 되었습니다.\n\n저는 처음 본 조원들에게 효율적인 업무 분담하는 과정이 큰 난관이었습니다. 조원들 간의 장점을 최대한 끌어낼 방법을 찾기 위해 노력했습니다. 장시간 소통을 통해 문제를 해결하는 것이 우선이었습니다. 작품 회의를 하며 여러 업무를 분배하는 과정에서 개인마다 각자 잘할 수 있는 일이 있다는 사실을 인지할 수 있었습니다. 이를 바탕으로, 디자인 경력이 있는 조원에게는 디자인 업무, 평소에 책 읽기를 좋아하고 글 쓰는 재주가 있는 조원에게는 계획서 작성, 플랫폼 구현경험이 있는 조원에게는 App 구성 등 효율적인 업무분담을 할 수 있었습니다.\n\n1년이라는 장기프로젝트를 진행하면서 가장 힘들었던 점은 시간과 공간의 제약이었습니다. 각자의 전공공부와 프로젝트를 병행하는 시간적인 제약과 각각 다른 학교 출신으로 공간적인 제약을 조율하는 것은 정말 힘든 일이었습니다. 프로젝트를 자율적으로 진행하다 보니 진행 상황이 더뎌지는 상황을 확인할 수 있었습니다. 저희는 주 1회 정기적으로 모여 개개인의 업무 상황을 보고하고 서로의 업무진행에 대해 피드백하기로 했습니다. 피드백을 통해 구체적인 메시지를 전달하여 서로가 학습하고 성장할 수 있게 만들었습니다.\n\n팀워크의 가치를 현대모비스에서 실행해보고 싶습니다. 협동 부서의 역할을 이해하고 소통하여 팀원 간, 부서 간의 징검다리가 되겠습니다."

In [81]:
interview_data.Question = interview_data.Question.apply(text_preprocess)
interview_data.Answer = interview_data.Answer.apply(text_preprocess)

In [ ]:
interview_data['Answer'][16]

"OO에서 주관하는 '제O기 사회리더 대학생 멘토링' 활동 때 팀워크를 발휘하여 공동의 목표 달성에 이바지한 경험이 있습니다. 저희는 고객과 헤어샵 사이 중개 플랫폼을 만들어보자는 목표를 설정하였습니다. 프로젝트에 고객수요조사, 플랫폼 설계 등 혼자 하기에는 무리가 있어서 팀을 구성하였고 저를 포함 5명이 팀을 이루게 되었습니다. 저는 처음 본 조원들에게 효율적인 업무 분담하는 과정이 큰 난관이었습니다. 조원들 간의 장점을 최대한 끌어낼 방법을 찾기 위해 노력했습니다. 장시간 소통을 통해 문제를 해결하는 것이 우선이었습니다. 작품 회의를 하며 여러 업무를 분배하는 과정에서 개인마다 각자 잘할 수 있는 일이 있다는 사실을 인지할 수 있었습니다. 이를 바탕으로, 디자인 경력이 있는 조원에게는 디자인 업무, 평소에 책 읽기를 좋아하고 글 쓰는 재주가 있는 조원에게는 계획서 작성, 플랫폼 구현경험이 있는 조원에게는 App 구성 등 효율적인 업무분담을 할 수 있었습니다. 1년이라는 장기프로젝트를 진행하면서 가장 힘들었던 점은 시간과 공간의 제약이었습니다. 각자의 전공공부와 프로젝트를 병행하는 시간적인 제약과 각각 다른 학교 출신으로 공간적인 제약을 조율하는 것은 정말 힘든 일이었습니다. 프로젝트를 자율적으로 진행하다 보니 진행 상황이 더뎌지는 상황을 확인할 수 있었습니다. 저희는 주 1회 정기적으로 모여 개개인의 업무 상황을 보고하고 서로의 업무진행에 대해 피드백하기로 했습니다. 피드백을 통해 구체적인 메시지를 전달하여 서로가 학습하고 성장할 수 있게 만들었습니다. 팀워크의 가치를 XX에서 실행해보고 싶습니다. 협동 부서의 역할을 이해하고 소통하여 팀원 간, 부서 간의 징검다리가 되겠습니다."

# 토크나이징

HuggingFace's BPEtokenizer

In [8]:
tokenize_data = interview_data.values.tolist()

In [82]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(
                limit_alphabet=10000,
                min_frequency=3,
                special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                continuing_subword_prefix="▁"
                )

tokenizer.pre_tokenizer = Whitespace()

tokenizer.train_from_iterator(tokenize_data, trainer)

In [83]:
tokenizer.get_vocab_size()

4012

In [84]:
tokenizer.save('/content/drive/Othercomputers/내 컴퓨터/Interview_Writer/BPE_tokenizer.json')

In [85]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file='/content/drive/Othercomputers/내 컴퓨터/Interview_Writer/BPE_tokenizer.json')

In [86]:
interview_data.Question = interview_data.Question.apply(fast_tokenizer.tokenize)
interview_data.Answer = interview_data.Answer.apply(fast_tokenizer.tokenize)

In [87]:
print(interview_data['Question'][1])
print(interview_data['Answer'][1])

['지원', '▁직무', '▁를', '선택한', '이유와', '해당직무에', '본인이', '적', '▁임', '▁자', '▁라고', '생각하는', '이유를', '기술해주십시오', '.']
['대학교', '2학년', '1', '▁학기', ',', '프로그래밍', '▁언어', '수업을', '통해', '영상처리', '▁를', '처음', '접', '▁하게', '되었습니다', '.', 'C', '++', '을', '이용해', '이미지를', '이', '▁진', '▁화', ',', '반', '▁전', '▁하는', '실습을', '하고', '원하는', '방식', '▁대로', '이미지를', '변', '▁형', '▁하는', '프로젝트를', '진행하며', '영상처리', '▁에', '대해', '관심을', '가지게', '되었습니다', '.', '추가로', '디지털', '영상처리', '수업을', '통해', 'M', '▁at', '▁L', '▁a', '▁b', '▁을', '이용해', '암', '▁을', '찾아내는', '실습', ',', '딥러닝을', '이용한', '물', '▁체', '▁인', '▁식', '실습', '▁과', '화', '▁질', '개선', '실습', '등을', '통해', '영상처리', '과정이', '흥미', '▁로운', '분야', '▁인', '것을', '깨달', '▁았고', '즐', '▁겁', '▁게', '일을', '할', '수', '있는', '유', '▁일', '▁한', '분야', '▁라고', '생각해', '해당', '직무를', '선택', '▁하게', '되었습니다', '.', '디지털', '영상처리', '수업을', '통해', 'M', '▁at', '▁L', '▁a', '▁b', '▁을', '이용한', '이', '▁진', '▁화', ',', '노', '▁이', '▁즈', '제', '▁거', '▁와', '같은', '영상처리', '기술', '▁들을', '배', '▁웠고', '학부', '연구생', '활동을', '통해', 'Python', ',', 'C', '#', '▁,', 'P

In [88]:
print(interview_data['Question'][16])
print(interview_data['Answer'][16])

['공동의', '목표를', '달성하기', '위해', '다른', '사람들과', '힘', '▁을', '합', '▁쳐', '노력', '▁했던', '경험을', '구체적으로', '기술하고', ',', '그', '경험을', '통해', '배운', '점을', '작성해', '주세요', '.']
['OO에서', '주관', '▁하는', "'", '제', '▁O', '▁기', '사회', '▁리', '▁더', '대학생', '멘토링', "'", '활동', '때', '팀워크를', '발휘하여', '공동의', '목표', '달성에', '이바지', '▁한', '경험이', '있습니다', '.', '저희', '▁는', '고객과', '헤', '▁어', '▁샵', '사', '▁이', '중', '▁개', '플랫폼을', '만들어', '▁보', '▁자는', '목표를', '설정하였습니다', '.', '프로젝트에', '고객', '▁수', '▁요', '▁조사', ',', '플랫폼', '설계', '등', '혼자', '하기', '▁에는', '무', '▁리가', '있어서', '팀을', '구성', '▁하였고', '저를', '포함', '5명', '▁이', '팀을', '이루', '▁게', '되었습니다', '.', '저는', '처음', '본', '조원', '▁들에게', '효율적인', '업무', '분담', '▁하는', '과정이', '큰', '난관', '▁이었습니다', '.', '조원', '▁들', '간의', '장', '▁점을', '최대한', '끌어', '▁낼', '방법을', '찾기', '위해', '노력했습니다', '.', '장', '▁시간', '소통을', '통해', '문제를', '해결하는', '것이', '우선', '▁이었습니다', '.', '작품', '회의를', '하며', '여러', '업무를', '분', '▁배', '▁하는', '과정에서', '개인', '▁마다', '각자', '잘', '▁할', '수', '있는', '일이', '있다는', '사실', '▁을', '인', '▁지', '▁할', '수', '있었습니다', '.', '이

HuggingFace's WordPiecetokenizer

In [15]:
tokenize_data = interview_data.values.tolist()

In [71]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
trainer = WordPieceTrainer(
                limit_alphabet=10000,
                min_frequency=3,
                special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
                )

tokenizer.pre_tokenizer = Whitespace()

tokenizer.train_from_iterator(tokenize_data, trainer)

In [72]:
tokenizer.get_vocab_size()

4012

In [75]:
tokenizer.save('/content/drive/Othercomputers/내 컴퓨터/Interview_Writer/WordPiece_tokenizer.json')

In [76]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file='/content/drive/Othercomputers/내 컴퓨터/Interview_Writer/WordPiece_tokenizer.json')

In [77]:
interview_data.Question = interview_data.Question.apply(fast_tokenizer.tokenize)
interview_data.Answer = interview_data.Answer.apply(fast_tokenizer.tokenize)

In [78]:
print(interview_data['Question'][1])
print(interview_data['Answer'][1])

['지원', '##직무', '##를', '선택한', '이유와', '해당직무에', '본인이', '적', '##임', '##자', '##라고', '생각하는', '이유를', '기술해주십시오', '.']
['대학교', '2학년', '1', '##학기', ',', '프로그래밍', '##언어', '수업을', '통해', '영상처리', '##를', '처음', '접', '##하게', '되었습니다', '.', 'C', '++', '을', '이용해', '이미지를', '이', '##진', '##화', ',', '반', '##전', '##하는', '실습을', '하고', '원하는', '방식', '##대로', '이미지를', '변', '##형', '##하는', '프로젝트를', '진행하며', '영상처리', '##에', '대해', '관심을', '가지게', '되었습니다', '.', '추가로', '디지털', '영상처리', '수업을', '통해', 'M', '##at', '##L', '##a', '##b', '##을', '이용해', '암', '##을', '찾아내는', '실습', ',', '딥러닝을', '이용한', '물', '##체', '##인', '##식', '실습', '##과', '화', '##질', '개선', '실습', '등을', '통해', '영상처리', '과정이', '흥미로', '##운', '분야', '##인', '것을', '깨달', '##았고', '즐', '##겁', '##게', '일을', '할', '수', '있는', '유', '##일', '##한', '분야', '##라고', '생각해', '해당', '직무를', '선택', '##하게', '되었습니다', '.', '디지털', '영상처리', '수업을', '통해', 'M', '##at', '##L', '##a', '##b', '##을', '이용한', '이', '##진', '##화', ',', '노', '##이', '##즈', '제', '##거', '##와', '같은', '영상처리', '기술', '##들을', '배', '##웠고', '학부', '연구

In [79]:
print(interview_data['Question'][16])
print(interview_data['Answer'][16])

['공동의', '목표를', '달성하기', '위해', '다른', '사람들과', '힘', '##을', '합', '##쳐', '노력', '##했던', '경험을', '구체적으로', '기술하고', ',', '그', '경험을', '통해', '배운', '점을', '작성해', '주세요', '.']
['OO에서', '주관', '##하는', "'", '제', '##O', '##기', '사회', '##리', '##더', '대학생', '멘토링', "'", '활동', '때', '팀워크를', '발휘하여', '공동의', '목표', '달성에', '이바지', '##한', '경험이', '있습니다', '.', '저희', '##는', '고객과', '헤', '##어', '##샵', '사', '##이', '중', '##개', '플랫폼을', '만들어', '##보', '##자는', '목표를', '설정하였습니다', '.', '프로젝트에', '고객', '##수', '##요', '##조사', ',', '플랫폼', '설계', '등', '혼자', '하기', '##에는', '무', '##리가', '있어서', '팀을', '구성', '##하였고', '저를', '포함', '5명', '##이', '팀을', '이루', '##게', '되었습니다', '.', '저는', '처음', '본', '조원', '##들에게', '효율적인', '업무', '분담', '##하는', '과정이', '큰', '난관', '##이었습니다', '.', '조원', '##들', '간의', '장점', '##을', '최대한', '끌어', '##낼', '방법을', '찾기', '위해', '노력했습니다', '.', '장', '##시간', '소통을', '통해', '문제를', '해결하는', '것이', '우선', '##이었습니다', '.', '작품', '회의를', '하며', '여러', '업무를', '분', '##배', '##하는', '과정에서', '개인', '##마다', '각자', '잘', '##할', '수', '있는', '일이', '있다는', '사실', '##을', '